# Complete MELD Training Pipeline - Friends Dating Simulator 🤖❤️

**Purpose:** Self-contained notebook for training LLaMA 3.1 on MELD dating simulator

**What this notebook does:**
1. 📚 Loads raw cleaned MELD data
2. 📝 Formats data with LLaMA 3.1 instruction template
3. 🎯 Fine-tunes LLaMA 3.1 with LoRA
4. ✅ Tests generation with FIXED parameters (no repetition, proper stopping)

**Key Features:**
- Everything in one place (no external data dependencies)
- Uses correct LLaMA 3.1 chat template
- Fixed generation function (proper EOS token, repetition penalty)
- Immediate testing after training

---

## 1. Setup and Configuration

In [ ]:
!pip3 install torch
!pip3 install pandas
!pip3 install numpy
!pip3 install tqdm
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install transformers
!pip3 install datasets
!pip3 install accelerate
!pip3 install bitsandbytes
!pip3 install tensorboard
!pip3 install pyyaml
!pip3 install peft
!pip3 install --upgrade ipywidgets traitlets ipykernel tqdm

In [ ]:
# Check environment
import sys
from pathlib import Path

# Add parent to path
if Path.cwd().name == 'MELD':
    sys.path.insert(0, str(Path.cwd().parent.parent))
    print("✓ Running from MELD directory")
else:
    print(f"⚠️  Current directory: {Path.cwd()}")
    print("Please run from notebooks/MELD/ directory")

In [ ]:
# Core imports
import torch
import json
import pandas as pd
import numpy as np
import random
import re
from datetime import datetime
from tqdm.notebook import tqdm

# Transformers and PEFT
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from datasets import Dataset, DatasetDict

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline

print("✓ All imports successful")

In [ ]:
# GPU Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
    
    # Clear cache
    torch.cuda.empty_cache()
else:
    print("⚠️  No GPU detected - training will be VERY slow")

## 2. Training Configuration

**⚠️ CUSTOMIZE THESE SETTINGS:**

In [ ]:
# ==================== CONFIGURATION ====================

# Model settings
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

# Data paths
DATA_PATH = "../../data/processed/MELD/meld_romantic_cleaned.csv"  # Raw cleaned data
OUTPUT_DIR = "../../checkpoints/dating_sim_meld_fixed"  # New output directory

# Training hyperparameters
CONFIG = {
    # Data
    'max_length': 512,
    'train_split': 0.9,
    
    # Training
    'num_epochs': 10,
    'batch_size': 1,
    'gradient_accumulation_steps': 8,
    'learning_rate': 2e-4,
    'warmup_steps': 100,
    'weight_decay': 0.01,
    
    # LoRA parameters
    'lora_r': 8,
    'lora_alpha': 16,
    'lora_dropout': 0.05,
    'lora_target_modules': ['q_proj', 'v_proj', 'k_proj', 'o_proj'],
    
    # Memory optimization
    'gradient_checkpointing': True,
    'fp16': True,
    'bf16': False,
    
    # Logging
    'logging_steps': 10,
    'eval_steps': 100,
    'save_steps': 500,
    'save_total_limit': 3,
}

print("Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Data: {DATA_PATH}")
print(f"  Output: {OUTPUT_DIR}")
print(f"  Epochs: {CONFIG['num_epochs']}")
print(f"  Effective batch size: {CONFIG['batch_size'] * CONFIG['gradient_accumulation_steps']}")

---
## 3. Load Raw Cleaned Data

In [ ]:
# Load cleaned data from notebook 01
print(f"Loading cleaned data from: {DATA_PATH}")
df = pd.read_csv(DATA_PATH)

print(f"✓ Loaded {len(df)} dialogue pairs")
print(f"\nColumns: {list(df.columns)}")
print(f"Dataset shape: {df.shape}")

# Display sample
print("\nSample data:")
df.head()

In [ ]:
# Data statistics
print("="*80)
print("Character Distribution")
print("="*80)
char_counts = df['character'].value_counts()
for char, count in char_counts.items():
    percentage = (count / len(df)) * 100
    print(f"{char:15s}: {count:5d} ({percentage:5.2f}%)")

print("\n" + "="*80)
print("Emotion Distribution")
print("="*80)
emotion_counts = df['emotion'].value_counts()
for emotion, count in emotion_counts.items():
    percentage = (count / len(df)) * 100
    print(f"{emotion:15s}: {count:5d} ({percentage:5.2f}%)")

---
## 4. Load Tokenizer for Formatting

In [ ]:
# Load LLaMA 3.1 tokenizer
print("Loading LLaMA 3.1 tokenizer for data formatting...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"✓ Tokenizer loaded: {tokenizer.__class__.__name__}")
print(f"  Special tokens: {tokenizer.special_tokens_map}")
print(f"  EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")

---
## 5. Define Character Personas & Scenarios

From notebook 02

In [ ]:
# Friends character personas
CHARACTER_PERSONAS = {
    'Chandler': {
        'description': "You are Chandler Bing from Friends. You are witty, sarcastic, and use humor as a defense mechanism to hide your vulnerability. You make jokes even in serious moments, but you're actually very caring and romantic underneath. You're self-deprecating and sometimes awkward, but loyal and loving to those close to you.",
        'traits': ['witty', 'sarcastic', 'defensive humor', 'secretly romantic', 'self-deprecating']
    },
    'Monica': {
        'description': "You are Monica Geller from Friends. You are organized, competitive, and love to be in control. You're nurturing and care deeply about the people in your life. You want commitment and stability in relationships. You can be intense but you're passionate about everything you do, from cooking to loving your partner.",
        'traits': ['organized', 'competitive', 'nurturing', 'wants commitment', 'passionate']
    },
    'Ross': {
        'description': "You are Ross Geller from Friends. You are intellectual, nerdy, and passionate about paleontology and science. You tend to overthink things and can be awkward in romantic situations. You're a hopeless romantic who believes in destiny and true love, but you often struggle to express your feelings properly.",
        'traits': ['intellectual', 'nerdy', 'overthinks', 'romantic but awkward', 'passionate']
    },
    'Rachel': {
        'description': "You are Rachel Green from Friends. You are fashion-focused, fun, and flirty. You're independent and career-driven, having grown from a spoiled daddy's girl to a confident professional. You value friendship and are loyal to those you care about. You're charming and know how to make people feel special.",
        'traits': ['fashion-focused', 'fun', 'flirty', 'independent', 'confident']
    },
    'Joey': {
        'description': "You are Joey Tribbiani from Friends. You are confident, charming, and simple in the best way. You love food (especially pizza and sandwiches) and you're famous for your catchphrase 'How you doin'?' You're a loyal friend and while you may not be the smartest, you have a big heart and know how to make people feel good about themselves.",
        'traits': ['confident', 'charming', 'simple', 'food-loving', 'loyal']
    },
    'Phoebe': {
        'description': "You are Phoebe Buffay from Friends. You are quirky, spiritual, and unconventionally wise. You're honest to a fault and say what's on your mind. You have a mysterious past but maintain an optimistic outlook. You're free-spirited and bring unique perspectives to every situation. You believe in karma, auras, and following your heart.",
        'traits': ['quirky', 'spiritual', 'honest', 'unconventional', 'optimistic']
    }
}

# Dating scenarios
DATING_SCENARIOS = [
    "You're on a casual coffee date at Central Perk, the cozy coffee shop.",
    "You're having a romantic dinner date at a nice restaurant.",
    "You're taking a walk together and having a deep conversation.",
    "You're hanging out at your apartment, enjoying each other's company.",
    "You're on a fun date doing something adventurous together.",
    "You're having a heart-to-heart conversation about your relationship.",
    "You're flirting and getting to know each other better.",
    "You're spending a quiet evening together, just talking and connecting.",
]

print(f"✓ Loaded {len(CHARACTER_PERSONAS)} character personas")
print(f"✓ Loaded {len(DATING_SCENARIOS)} dating scenarios")

---
## 6. Format Data with LLaMA 3.1 Instruction Template

From notebook 02

In [ ]:
def generate_scenario(row):
    """
    Generate deterministic scenario based on dialogue_id.
    """
    random.seed(int(row['dialogue_id']))
    return random.choice(DATING_SCENARIOS)

def format_instruction_prompt(row):
    """
    Format dialogue pair using LLaMA 3.1 chat template.
    
    Uses tokenizer.apply_chat_template() with:
    - System prompt: persona + scenario + emotion
    - User message: conversation context
    - Assistant response: character's reply
    
    Returns fully formatted LLaMA 3.1 prompt string.
    """
    character = row['character']
    emotion = row['emotion']
    context = row['context']
    response = row['response']
    
    # Get persona description
    persona_desc = CHARACTER_PERSONAS.get(character, {}).get(
        'description',
        f"You are {character} from Friends."
    )
    
    # Generate scenario
    scenario = generate_scenario(row)
    
    # Build system prompt
    system_content = f"""{persona_desc}

Scenario: {scenario}
The user seems to be feeling: {emotion}"""
    
    # User message with context
    user_content = f"Conversation:\n{context}"
    
    # Build messages for chat template
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": response}
    ]
    
    # Format with LLaMA 3.1 chat template
    formatted = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    return formatted

print("✓ Formatting functions defined")

In [ ]:
# Test formatting with a sample
print("="*80)
print("Sample Formatted Instruction (LLaMA 3.1 Format)")
print("="*80)
sample_row = df.iloc[0]
sample_prompt = format_instruction_prompt(sample_row)
print(sample_prompt[:600] + "..." if len(sample_prompt) > 600 else sample_prompt)
print("\n" + "="*80)

In [ ]:
# Apply formatting to all rows
print("Formatting all dialogue pairs with LLaMA 3.1 chat template...")
df['text'] = df.apply(format_instruction_prompt, axis=1)
print(f"✓ Formatted {len(df)} dialogue pairs")

# Statistics
df['text_length'] = df['text'].apply(len)
print(f"\nFormatted text length statistics:")
print(f"  Mean: {df['text_length'].mean():.0f} characters")
print(f"  Median: {df['text_length'].median():.0f} characters")
print(f"  Min: {df['text_length'].min()} characters")
print(f"  Max: {df['text_length'].max()} characters")

---
## 7. Create HuggingFace Dataset

In [ ]:
# Convert to HuggingFace Dataset
dataset_df = df[['text']].copy()
dataset = Dataset.from_pandas(dataset_df)

# Train/validation split
train_test = dataset.train_test_split(
    test_size=1-CONFIG['train_split'],
    seed=42
)

train_dataset = train_test['train']
val_dataset = train_test['test']

print(f"✓ Dataset created")
print(f"  Train samples: {len(train_dataset)}")
print(f"  Validation samples: {len(val_dataset)}")
print(f"\nExample training sample (first 400 chars):")
print(train_dataset[0]['text'][:400] + "...")

---
## 8. Load Model with LoRA

In [ ]:
# Set padding token for tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    print("✓ Set pad_token to eos_token")

print(f"Tokenizer info:")
print(f"  Pad token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"  EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")

In [ ]:
# Load base model
print(f"\nLoading model: {MODEL_NAME}")
print("This may take a few minutes...\n")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if CONFIG['fp16'] else torch.bfloat16 if CONFIG['bf16'] else torch.float32,
    device_map='auto',
    trust_remote_code=True
)

print("✓ Model loaded")
total_params = sum(p.numel() for p in model.parameters())
print(f"  Total parameters: {total_params:,}")
print(f"  Size: ~{total_params * 2 / 1e9:.2f} GB (FP16)")

In [ ]:
# Configure LoRA
if CONFIG['gradient_checkpointing']:
    model.gradient_checkpointing_enable()
    print("✓ Gradient checkpointing enabled")

lora_config = LoraConfig(
    r=CONFIG['lora_r'],
    lora_alpha=CONFIG['lora_alpha'],
    target_modules=CONFIG['lora_target_modules'],
    lora_dropout=CONFIG['lora_dropout'],
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

print("\n✓ LoRA applied")
model.print_trainable_parameters()

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nMemory for trainable params: ~{trainable_params * 2 / 1e9:.3f} GB (FP16)")

---
## 9. Tokenize Training Data

In [ ]:
def tokenize_function(examples):
    """
    Tokenize formatted dialogues.
    """
    tokenized = tokenizer(
        examples,
        truncation=True,
        max_length=CONFIG['max_length'],
        padding='max_length',
        return_tensors='pt'
    )
    
    # For causal LM, labels = input_ids
    tokenized['labels'] = tokenized['input_ids'].clone()
    
    return tokenized

print("✓ Tokenization function defined")

In [ ]:
# Tokenize datasets
print("Tokenizing datasets...")

train_texts = [train_dataset[i]['text'] for i in range(len(train_dataset))]
val_texts = [val_dataset[i]['text'] for i in range(len(val_dataset))]

train_tokenized = tokenize_function(train_texts)
val_tokenized = tokenize_function(val_texts)

# Create torch datasets
class DialogueDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __len__(self):
        return len(self.encodings['input_ids'])
    
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

train_torch_dataset = DialogueDataset(train_tokenized)
val_torch_dataset = DialogueDataset(val_tokenized)

print(f"✓ Tokenization complete")
print(f"  Train samples: {len(train_torch_dataset)}")
print(f"  Val samples: {len(val_torch_dataset)}")

---
## 10. Configure Training

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    
    # Training
    num_train_epochs=CONFIG['num_epochs'],
    per_device_train_batch_size=CONFIG['batch_size'],
    per_device_eval_batch_size=CONFIG['batch_size'],
    gradient_accumulation_steps=CONFIG['gradient_accumulation_steps'],
    
    # Optimization
    learning_rate=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay'],
    warmup_steps=CONFIG['warmup_steps'],
    lr_scheduler_type='cosine',
    
    # Memory optimization
    fp16=CONFIG['fp16'],
    bf16=CONFIG['bf16'],
    gradient_checkpointing=CONFIG['gradient_checkpointing'],
    
    # Logging and saving
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=CONFIG['logging_steps'],
    eval_steps=CONFIG['eval_steps'],
    save_steps=CONFIG['save_steps'],
    save_total_limit=CONFIG['save_total_limit'],
    eval_strategy='steps',
    save_strategy='steps',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    
    # Other
    report_to='tensorboard',
    remove_unused_columns=False,
)

print("Training configuration:")
print(f"  Output dir: {OUTPUT_DIR}")
print(f"  Effective batch size: {CONFIG['batch_size'] * CONFIG['gradient_accumulation_steps']}")
total_steps = len(train_torch_dataset) // (CONFIG['batch_size'] * CONFIG['gradient_accumulation_steps']) * CONFIG['num_epochs']
print(f"  Total steps: {total_steps}")
print(f"  Mixed precision: {'FP16' if CONFIG['fp16'] else 'BF16' if CONFIG['bf16'] else 'FP32'}")

In [ ]:
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Early stopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.001
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_torch_dataset,
    eval_dataset=val_torch_dataset,
    data_collator=data_collator,
    callbacks=[early_stopping],
)

print("✓ Trainer initialized with early stopping")

---
## 11. Train Model 🚀

In [ ]:
# Clear GPU cache before training
import gc
gc.collect()
torch.cuda.empty_cache()

print("Starting training...")
print(f"Monitor progress: tensorboard --logdir {OUTPUT_DIR}/logs")
print()

In [ ]:
# Train!
train_result = trainer.train()

print("\n" + "="*80)
print("Training Complete! 🎉")
print("="*80)
print(f"Training loss: {train_result.training_loss:.4f}")
print(f"Training time: {train_result.metrics['train_runtime']:.2f} seconds")

---
## 12. Save Model

In [ ]:
# Save final model
final_model_path = f"{OUTPUT_DIR}/final"
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"✓ Model saved to: {final_model_path}")

# Save training metrics
metrics_path = f"{OUTPUT_DIR}/training_metrics.json"
with open(metrics_path, 'w') as f:
    json.dump(train_result.metrics, f, indent=2)

print(f"✓ Metrics saved to: {metrics_path}")

---
## 13. Test Generation with FIXED Parameters 🔧

Test the trained model with corrected generation function

In [ ]:
# Set model to eval mode
model.eval()

def generate_response_fixed(character, user_input, emotion="neutral", scenario=None, max_new_tokens=50, temperature=0.7, top_p=0.9):
    """
    Generate response with FIXED parameters for proper stopping.
    """
    # Get persona
    persona_desc = CHARACTER_PERSONAS.get(
        character, {}).get('description',
        f"You are {character} from Friends."
    )
    
    # Use provided scenario or default
    if scenario is None:
        scenario = "You're on a casual coffee date at Central Perk, the cozy coffee shop."
    
    # Build system prompt
    system_content = f"""{persona_desc}

Scenario: {scenario}
The user seems to be feeling: {emotion}"""
    
    # User message
    user_content = f"Conversation:\n{user_input}"
    
    # Build messages for LLaMA 3.1 chat template
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]
    
    # Apply chat template WITH generation prompt
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    input_length = inputs['input_ids'].shape[1]
    
    # Generate with FIXED parameters
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,  # FIX: Correct EOS token
            repetition_penalty=1.2,  # FIX: Add repetition penalty
            no_repeat_ngram_size=3   # FIX: Prevent 3-gram repetition
        )
    
    # Extract only the generated tokens
    generated_tokens = outputs[0][input_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    # Safety net: Remove any accidental speaker tokens
    response = re.sub(r'<[^>]+>\s*', '', response)
    response = ' '.join(response.split())
    
    return response

print("✓ FIXED generation function ready")

In [ ]:
# Test with different Friends characters
test_cases = [
    ("Chandler", "How are you doing today?", "joy"),
    ("Monica", "Want to grab dinner?", "neutral"),
    ("Ross", "Tell me about your interests", "neutral"),
    ("Rachel", "You look nice today", "joy"),
    ("Joey", "How you doin'?", "neutral"),
    ("Phoebe", "What do you believe in?", "neutral"),
]

print("Testing FIXED generation with newly trained model:\n")
print("="*80)

for character, user_input, emotion in test_cases:
    response = generate_response_fixed(character, user_input, emotion)
    print(f"Character: {character}")
    print(f"User ({emotion}): {user_input}")
    print(f"Response: {response}")
    print("-"*80)

---
## 14. Summary

### What This Notebook Did:

1. ✅ Loaded raw cleaned MELD data
2. ✅ Formatted with correct LLaMA 3.1 chat template
3. ✅ Fine-tuned LLaMA 3.1 with LoRA
4. ✅ Tested generation with FIXED parameters

### Key Fixes Applied:

- ✅ Correct EOS token ID (`tokenizer.eos_token_id`)
- ✅ Repetition penalty (1.2) + no_repeat_ngram_size (3)
- ✅ Lower temperature (0.7)
- ✅ Reduced max_new_tokens (50)
- ✅ Token-based extraction
- ✅ Speaker token stripping

### Expected Results:

Responses should now:
- Be single-turn only
- Have no repetition
- Have no speaker tokens
- Stop cleanly at sentence end

### Next Steps:

1. **If generation works correctly:**
   - Use this trained model for evaluation
   - Update evaluation notebook 04 with same fixed generation
   - Run full evaluation on checkpoints

2. **Monitor training:**
   ```bash
   tensorboard --logdir {OUTPUT_DIR}/logs
   ```

3. **Files generated:**
   - Final model: `{OUTPUT_DIR}/final/`
   - Checkpoints: `{OUTPUT_DIR}/checkpoint-*/`
   - Training metrics: `{OUTPUT_DIR}/training_metrics.json`
   - TensorBoard logs: `{OUTPUT_DIR}/logs/`